# `metadsl`

...is a framework for creating domain specific languages in Python.

In [1]:
from metadsl_all import *

## LLVM
In this demo, we will show the current work on compiling to an LLVM function, using LLVM lite. 

In [15]:
import metadsl_rewrite
import metadsl as m
import metadsl_core as mc
import metadsl_llvm as ml
import metadsl_visualize

### Simple function

In [4]:
int_type = ml.Type.create_int(32)
one = ml.ValueExpr.from_value(ml.Value.constant(int_type, 1))

mod_ref = ml.ModRef.create("add")

@ml.llvm_fn(mod_ref, ml.FnType.create(int_type, int_type, int_type))
@mc.FunctionTwo.from_fn
def add(l: ml.ValueExpr, r: ml.ValueExpr) -> ml.ValueExpr:
    return l + r + one

real_fn = metadsl_rewrite.execute(ml.compile_functions(mod_ref, ml.to_llvm(add)))
assert real_fn(10, 11) == 22

Typez(definitions=None, nodes=[PrimitiveNode(id='620784253559051387', type='str', repr='add'), PrimitiveNode(id='3897048290446240432', type='function', repr='<function add at 0x1124b00d0>'), PrimitiveNode(id='-5076811030180719980', type='int', repr='32'), PrimitiveNode(id='-4262466165513757285', type='int', repr='1'), CallNode(id='-1259645767813234200', function='ModRef.create\n', type_params=None, args=['620784253559051387'], kwargs=None), CallNode(id='4216686198697142956', function='FunctionTwo.from_fn\n', type_params=None, args=['3897048290446240432'], kwargs=None), CallNode(id='896411822270447236', function='Type.create_int\n', type_params=None, args=['-5076811030180719980'], kwargs=None), CallNode(id='6920011226272107764', function='FnType.create\n', type_params=None, args=['896411822270447236', '896411822270447236', '896411822270447236'], kwargs=None), CallNode(id='900348640671044205', function='llvm_fn_2\n', type_params=None, args=['4216686198697142956', '-1259645767813234200', 

## Basic Types

Here we demos some of the basic types. 

For example, we can try to look at this function:

```python
def fn(x):
    if x > 10:
        return x + 100
    return x - 100
```

In [12]:
@mc.FunctionOne.from_fn
def fn(x: mc.Integer) -> mc.Integer:
    cond = x > mc.Integer.from_int(10)
    true = x + mc.Integer.from_int(100)
    false = x - mc.Integer.from_int(100)
    return cond.if_(true, false)

# Numpy

**No longer working below this line**

In [ ]:
from metadsl_all import *

Here, we show a NumPy API compatible DSL that we have started writing. We can create a symbolic expression and then execute it. The widget shows the different stages of execution:

In [2]:
N = 10

(arange(N) + arange(N))[5].to_ndarray()
expr

NameError: name 'arange' is not defined

Now, we can add an optimization replacement, so that we do the getitem before adding the two expressions:

In [3]:
@register_optimize
@rule
def optimize_getitem_add(l: NDArray, r: NDArray, idx: IndxType) -> R[NDArray]:
    return (
        # expression to match against
        (l + r)[idx],
        # expression to replace it with
        l[idx] + r[idx]
    )

NameError: name 'register_optimize' is not defined

Now, when we look at the execution steps, we see that it is doing each getitem before adding, which should be more efficient:

In [4]:
expr

NameError: name 'expr' is not defined

This shows how we can break up the NumPy API into different layers, all of which are extensible:

1. A compatibility layer that works like the existing NumPy API, except isn't limited to the Python types of the current API
2. A type safe version of this API. The conversion between the compatability layer and this layer is extensible, so that third party authors can add new conversion between their own Python objects and the typed representation.
3. A backend layer that translates either back to Python calls or source code, or to other targets like LLVM or Tensorflow.

The key is that all these layers are composable, so you could have different frontends for any of them or add your own. This is all done through a typed replacement system that is compatible with static analysis using MyPy.

`metadsl` is the glue that allows us to define this API in a way that enables greater greater collaboration. Since each layer has it's own well defined specification, different projects can extend and target these seperate layers, while minimizing the need to explicitly opt in to collaborate. The goal is to enable greater reuse, interoperability, and extensability within the Python scientific ecosystem.